# Dataset description
* 32 participants [s01 to s32]
* 40 pieces of music
* 40 channels out of which the first 32 are relevant 
* 8064 readings recorded at 128Hz


In [10]:
from deap_pre_process import *

In [11]:
eeg1d,eeg2d,val,aro = get_subject_data(subject_no=1,media_no=[0,1,2,3])

[+] Reading subject: s01 Media: [0, 1, 2, 3]


In [12]:
len(eeg2d[0].reshape(-1,9,9,1))

7680

In [13]:
eeg2d[0].shape

(7680, 9, 9)

In [14]:
val[0],aro[0]

(1, 1)

# Making the dummy network


In [15]:
from rnn_function import *
from cnn_function import *
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9534552419903780470
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13043372571722460657
physical_device_desc: "device: XLA_CPU device"
]


In [17]:
#Constant :
WINDOW_SIZE = 128
#For rnn layers
INPUT_NODES = 32
HIDDEN_ST = 32
HOLD_PROBA = 0.5
RNN_INPUT_SIZE = 32

#For cnn layers
CONV_IP_SHAPE = [-1,9,9,1]
CONV1_KERNEL = [4,4,1,32]
CONV2_KERNEL = [4,4,32,64]
CONV3_KERNEL = [4,4,64,128]
REDUCTION_KERNEL = [1,1,128*WINDOW_SIZE,13]

#For dnn layers (for the paper code)
N_FC_IN = 1024
N_FC_OUT= 1024

#FOR PREDICTIONS
N_LABELS = 2
LEARNING_RATE = 0.001

In [18]:
#FUNCTION FORM https://github.com/ynulonger/ijcnn/blob/master/cv.py
#FOR SHAPING THE INPUT OF LSTM
def apply_fully_connect(x, x_size, fc_size):
    fc_weight = init_weight([x_size, fc_size])
    fc_bias = init_bias([fc_size])
    return tf.nn.elu(tf.add(tf.matmul(x, fc_weight), fc_bias))

#FOR READOUT OF THE FINAL LAYER
def apply_readout(x, x_size, readout_size):
    readout_weight = init_weight([x_size, readout_size])
    readout_bias = init_bias([readout_size])
    return tf.add(tf.matmul(x, readout_weight), readout_bias)

In [19]:
#Resetting tf default graph
tf.reset_default_graph()

#CNN placeholders and model variables
x_data_cnn = tf.placeholder(dtype = tf.float32,shape=(None,9,9),name='cnn_data')
x_image = tf.reshape(x_data_cnn,CONV_IP_SHAPE,name='cnn_image')

#LAYERS
conv1 = conv_layer(x_image,CONV1_KERNEL,name="convo_l1")
conv2 = conv_layer(conv1,CONV2_KERNEL,name="convo_l2")
conv3 = conv_layer(conv2,CONV3_KERNEL,name="convo_l3")

#Depth variable
depth = tf.reshape(conv3,[-1,9,9,128* WINDOW_SIZE])

#Size reduction conv layer
reduced_data = conv_layer(depth,REDUCTION_KERNEL,name="conv_reduced")

#Flattening convolution layer
shape = reduced_data.get_shape().as_list()
final_flat = tf.reshape(reduced_data, [-1, shape[1] * shape[2] * shape[3]])
cnn_out_fuse = final_flat

#RNN placeholder and model variables
x_data_rnn = tf.placeholder(dtype = tf.float32,shape=(None,WINDOW_SIZE,INPUT_NODES),name='rnn_data') #TIME_STEP = WINDOW_SIZE
keep_prob = tf.placeholder(dtype=tf.float32)

#Shaping the RNN input
shape = x_data_rnn.get_shape().as_list()
rnn_in_flat = tf.reshape(x_data_rnn, [-1, shape[2]]) #[batch_size*n_time_step, n_electrode]
rnn_fc_in = apply_fully_connect(rnn_in_flat, shape[2], N_FC_IN) #[batch_size*n_time_step, n_electrode]
lstm_in = tf.reshape(rnn_fc_in, [-1,WINDOW_SIZE,N_FC_IN]) #[batch_size, n_time_step, n_fc_in]

#Making cells
cell1 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cell2 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cells = [cell1,cell2]
final_cell = create_Stacked_cell(cells)

#Laying out the network
output, states = create_RNN(final_cell,lstm_in)

#Unstacking the output
output = tf.unstack(tf.transpose(output, [1, 0, 2]), name='lstm_out')

#Selecting the final output form the layer
rnn_output = output[-1] #[batch, fc_size]

#Shaping the output
lstm_fc_out = dnn_layer(rnn_output,N_FC_OUT) #[batch_size, n_fc_out]
lstm_fc_drop = tf.nn.dropout(lstm_fc_out, keep_prob)

#Placeholder for true values
y_true = tf.placeholder(dtype=tf.float32,shape=(None,N_LABELS)) #[Valance, Arousal]

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [20]:
#Flattening the layer
final_connected_layer = tf.concat([cnn_out_fuse, lstm_fc_drop], axis=1)
shape = final_connected_layer.get_shape().as_list()
print("\nfuse_cnn_rnn:", shape)


fuse_cnn_rnn: [None, 2077]


In [21]:
#Creating output variables 
y_ = apply_readout(final_connected_layer, shape[1], N_LABELS)
y_pred = tf.argmax(tf.nn.softmax(y_), 1, name="y_pred")
y_posi = tf.nn.softmax(y_, name="y_posi")

#Making cost functions
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y_true), name='loss')
train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [22]:
# Saver and variable initializer
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# REMAINING:
* Session

In [24]:
eeg2d[0].shape

(7680, 9, 9)

In [25]:
eeg1d[0].shape

(7680, 32)

In [26]:
from time import time

In [27]:
with tf.Session() as sess:
    sess.run(init)
    t1=time()
    eeg1d,eeg2d,val,aro = get_subject_data(subject_no=1,media_no=[1])
    t2=time()
    for i in range(0,7680,WINDOW_SIZE):
        print("WINDOW FROM",i," TO ",i+WINDOW_SIZE)
        pred = np.array([val[0],aro[0]]).reshape(1,2)
        x_rnn = eeg1d[0][i:i+WINDOW_SIZE].reshape(-1,WINDOW_SIZE,32)
        x_cnn = eeg2d[0][i:i+WINDOW_SIZE]

        sess.run(train, feed_dict = {keep_prob:0.5,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: pred})
    
    t3=time()    

[+] Reading subject: s01 Media: [1]
WINDOW FROM 0  TO  128
WINDOW FROM 128  TO  256
WINDOW FROM 256  TO  384
WINDOW FROM 384  TO  512
WINDOW FROM 512  TO  640
WINDOW FROM 640  TO  768
WINDOW FROM 768  TO  896
WINDOW FROM 896  TO  1024
WINDOW FROM 1024  TO  1152
WINDOW FROM 1152  TO  1280
WINDOW FROM 1280  TO  1408
WINDOW FROM 1408  TO  1536
WINDOW FROM 1536  TO  1664
WINDOW FROM 1664  TO  1792
WINDOW FROM 1792  TO  1920
WINDOW FROM 1920  TO  2048
WINDOW FROM 2048  TO  2176
WINDOW FROM 2176  TO  2304
WINDOW FROM 2304  TO  2432
WINDOW FROM 2432  TO  2560
WINDOW FROM 2560  TO  2688
WINDOW FROM 2688  TO  2816
WINDOW FROM 2816  TO  2944
WINDOW FROM 2944  TO  3072
WINDOW FROM 3072  TO  3200
WINDOW FROM 3200  TO  3328
WINDOW FROM 3328  TO  3456
WINDOW FROM 3456  TO  3584
WINDOW FROM 3584  TO  3712
WINDOW FROM 3712  TO  3840
WINDOW FROM 3840  TO  3968
WINDOW FROM 3968  TO  4096
WINDOW FROM 4096  TO  4224
WINDOW FROM 4224  TO  4352
WINDOW FROM 4352  TO  4480
WINDOW FROM 4480  TO  4608
WINDOW FR

In [28]:
print("Import time:",t2-t1)
print("Process time:",t3-t2)

Import time: 0.26616525650024414
Process time: 27.142936944961548
